In [1]:
import sys
from pathlib import Path
from copy import deepcopy
def find_project_root(start_path: Path = Path.cwd(), marker: str = 'pyproject.toml') -> Path:
    current_path = start_path.resolve()
    for parent in [current_path] + list(current_path.parents):
        if (parent / marker).exists():
            return parent
        
def add_project_root_to_sys_path(marker: str = 'pyproject.toml'):
    project_root = find_project_root(marker=marker)
    if str(project_root) not in sys.path:
        sys.path.insert(0, str(project_root))

add_project_root_to_sys_path()


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
import numpy as np
import copy

from src.asym_ensembles.data_loaders import load_california_housing
from src.asym_ensembles.modeling.training import set_global_seed, train_one_model
from src.asym_ensembles.modeling.models import WMLP

In [3]:
train_ds, val_ds, test_ds = load_california_housing()
subset_size = 2000
train_ds_small, _ = random_split(train_ds, [subset_size, len(train_ds) - subset_size])

batch_size = 64
train_loader = DataLoader(train_ds_small, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)

in_features = train_ds.tensors[0].shape[1]
hidden_dim = 16
out_features = 1
num_layers = 4

In [4]:
seed_value = 123


mask_params = {
    0: {
        "mask_constant": 1,
        "mask_type": "random_subsets",
        "do_normal_mask": True,
        "num_fixed": 2,
    },
    1: {
        "mask_constant": 1,
        "mask_type": "random_subsets",
        "do_normal_mask": True,
        "num_fixed": 3,
    },
    2: {
        "mask_constant": 1,
        "mask_type": "random_subsets",
        "do_normal_mask": True,
        "num_fixed": 3,
    },
    3: {
        "mask_constant": 1,
        "mask_type": "random_subsets",
        "do_normal_mask": True,
        "num_fixed": 3,
    },
}
set_global_seed(seed_value)
wmlp1 = WMLP(in_features, hidden_dim, out_features, num_layers, mask_params, norm=None)



In [5]:
# set_global_seed(seed_value + 1)
wmlp2 = WMLP(in_features, hidden_dim, out_features, num_layers, mask_params, norm=None)

In [6]:
first_layer10 = deepcopy(wmlp1.lins[0])  # SparseLinear
first_layer20 = deepcopy(wmlp2.lins[0])

In [7]:
first_layer10.mask == first_layer20.mask

tensor([[True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True]])

In [8]:
first_layer10.weight * (1 - first_layer10.mask) == first_layer20.weight * (1 - first_layer20.mask)

tensor([[True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True]])

In [9]:
first_layer10.weight * first_layer10.mask == first_layer20.weight * first_layer20.mask

tensor([[False,  True, False, False, False, False,  True, False],
        [False, False,  True, False, False,  True, False, False],
        [ True, False, False, False, False, False,  True, False],
        [False, False,  True, False, False, False,  True, False],
        [False, False, False, False, False, False,  True,  True],
        [False,  True, False, False, False,  True, False, False],
        [ True, False,  True, False, False, False, False, False],
        [False, False, False, False, False,  True,  True, False],
        [False, False, False,  True, False,  True, False, False],
        [False, False, False, False, False, False,  True,  True],
        [False,  True,  True, False, False, False, False, False],
        [False,  True, False, False,  True, False, False, False],
        [False, False, False,  True, False, False,  True, False],
        [ True,  True, False, False, False, False, False, False],
        [False, False, False, False, False, False,  True,  True],
        [F

In [10]:
criterion = nn.MSELoss()

optimizer1 = torch.optim.AdamW(wmlp1.parameters(), lr=1e-3, weight_decay=0.0)
optimizer2 = torch.optim.AdamW(wmlp2.parameters(), lr=1e-3, weight_decay=0.0)

def train_for_epochs(model, optimizer, train_loader, epochs=5):
    device = "cpu"
    model.to(device)
    for epoch in range(epochs):
        model.train()
        for Xb, yb in train_loader:
            Xb, yb = Xb.to(device), yb.to(device)
            optimizer.zero_grad()
            preds = model(Xb)
            loss = criterion(preds, yb)
            loss.backward()
            optimizer.step()
    return model

train_for_epochs(wmlp1, optimizer1, train_loader, epochs=5)
train_for_epochs(wmlp2, optimizer2, train_loader, epochs=5)

WMLP(
  (lins): ModuleList(
    (0-3): 4 x SparseLinear()
  )
  (activation): GELU(approximate='none')
  (flatten): Flatten(start_dim=1, end_dim=-1)
)

In [11]:
first_layer1 = wmlp1.lins[0]  # SparseLinear
first_layer2 = wmlp2.lins[0]

In [12]:
first_layer1.mask == first_layer2.mask

tensor([[True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True]])

In [13]:
first_layer1.weight * (1 - first_layer1.mask) == first_layer2.weight * (1 - first_layer2.mask)

tensor([[True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True]])

In [14]:
first_layer1.weight * first_layer1.mask == first_layer2.weight * first_layer2.mask

tensor([[False,  True, False, False, False, False,  True, False],
        [False, False,  True, False, False,  True, False, False],
        [ True, False, False, False, False, False,  True, False],
        [False, False,  True, False, False, False,  True, False],
        [False, False, False, False, False, False,  True,  True],
        [False,  True, False, False, False,  True, False, False],
        [ True, False,  True, False, False, False, False, False],
        [False, False, False, False, False,  True,  True, False],
        [False, False, False,  True, False,  True, False, False],
        [False, False, False, False, False, False,  True,  True],
        [False,  True,  True, False, False, False, False, False],
        [False,  True, False, False,  True, False, False, False],
        [False, False, False,  True, False, False,  True, False],
        [ True,  True, False, False, False, False, False, False],
        [False, False, False, False, False, False,  True,  True],
        [F

In [15]:
first_layer1.weight * (1 - first_layer1.mask) == first_layer10.weight * (1 - first_layer10.mask)

tensor([[True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True]])

In [16]:
first_layer1.weight * (first_layer1.mask) == first_layer10.weight * (first_layer10.mask)

tensor([[False,  True, False, False, False, False,  True, False],
        [False, False,  True, False, False,  True, False, False],
        [ True, False, False, False, False, False,  True, False],
        [False, False,  True, False, False, False,  True, False],
        [False, False, False, False, False, False,  True,  True],
        [False,  True, False, False, False,  True, False, False],
        [ True, False,  True, False, False, False, False, False],
        [False, False, False, False, False,  True,  True, False],
        [False, False, False,  True, False,  True, False, False],
        [False, False, False, False, False, False,  True,  True],
        [False,  True,  True, False, False, False, False, False],
        [False,  True, False, False,  True, False, False, False],
        [False, False, False,  True, False, False,  True, False],
        [ True,  True, False, False, False, False, False, False],
        [False, False, False, False, False, False,  True,  True],
        [F

In [17]:
first_layer2.weight * (1 - first_layer2.mask) == first_layer20.weight * (1 - first_layer20.mask)

tensor([[True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True]])

In [18]:
first_layer2.weight * (first_layer2.mask) == first_layer20.weight * (first_layer20.mask)

tensor([[False,  True, False, False, False, False,  True, False],
        [False, False,  True, False, False,  True, False, False],
        [ True, False, False, False, False, False,  True, False],
        [False, False,  True, False, False, False,  True, False],
        [False, False, False, False, False, False,  True,  True],
        [False,  True, False, False, False,  True, False, False],
        [ True, False,  True, False, False, False, False, False],
        [False, False, False, False, False,  True,  True, False],
        [False, False, False,  True, False,  True, False, False],
        [False, False, False, False, False, False,  True,  True],
        [False,  True,  True, False, False, False, False, False],
        [False,  True, False, False,  True, False, False, False],
        [False, False, False,  True, False, False,  True, False],
        [ True,  True, False, False, False, False, False, False],
        [False, False, False, False, False, False,  True,  True],
        [F

In [1]:
from sklearn.datasets import fetch_california_housing, fetch_covtype, load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from torch.utils.data import TensorDataset
data = fetch_california_housing()

In [3]:
print(data.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

:Number of Instances: 20640

:Number of Attributes: 8 numeric, predictive attributes and the target

:Attribute Information:
    - MedInc        median income in block group
    - HouseAge      median house age in block group
    - AveRooms      average number of rooms per household
    - AveBedrms     average number of bedrooms per household
    - Population    block group population
    - AveOccup      average number of household members
    - Latitude      block group latitude
    - Longitude     block group longitude

:Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived from the 1990 U.S. census, using one row per ce